El modelo por excelencia al realizar una recomendacion es la similitud del coseno, de ahi partis una vez tengas vectorizadas tus palabras. Vos podes usar el que quieras, siempre y cuando cumpla la funcion de recomendar.

hagas el 

In [165]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

In [217]:
df_movies=pd.read_csv('../DatosProcesados/movies.csv')

In [218]:
chosen_columns = ["title", 'genres', "overview"]
df_movies = df_movies[chosen_columns]

In [220]:
df_movies.sample(3)

,title,genres,overview
22917,Trancers 3: Deth Lives,"Action, Science Fiction, Thriller","A time traveling cop, Jack Deth, from the futu..."
37139,Island of Death,"Horror, Thriller",A British couple on a break on a small Greek I...
18586,Dragon Age: Redemption,NaN,Written by and starring Felicia Day (The Guild...


In [221]:
df_movies['combinado'] = df_movies['genres'] +' '+ df_movies['title'] + ' ' + df_movies['overview']


In [222]:
df_movies['combinado'] = df_movies['combinado'].apply( lambda x: re.sub(r'[^\w\s]', '', x) if pd.notnull(x) else '' )


In [223]:
df_movies['combinado'][1]

'Adventure Fantasy Family Jumanji When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world they unwittingly invite Alan  an adult whos been trapped inside the game for 26 years  into their living room Alans only hope for freedom is to finish the game which proves risky as all three find themselves running from giant rhinoceroses evil monkeys and other terrifying creatures'

In [224]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies['combinado'])

In [225]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [226]:
cosine_sim

array([[1.        , 0.00627464, 0.00218974, ..., 0.        , 0.        ,
        0.        ],
       [0.00627464, 1.        , 0.00852528, ..., 0.        , 0.        ,
        0.        ],
       [0.00218974, 0.00852528, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [227]:
item_similarity=pd.DataFrame(cosine_sim,index=df_movies['title'],columns=df_movies['title'])
item_similarity

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,"Heat: Obsessive master thief, Neil McCauley leads a top-notch crew on...","Sabrina: An ugly duckling having undergone a remarkable change, still harbors...",Tom and Huck,Sudden Death,GoldenEye,...,St. Michael Had a Rooster,House of Horrors,Shadow of the Blair Witch,The Burkittsville 7,Caged Heat 3000,"Robin Hood: Yet another version of the classic epic, with enough variation...",Century of Birthing,"Betrayal: When one of her hits goes wrong, a professional assassin...",Satan Triumphant,Queerama
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.006275,0.002190,0.000860,0.000914,0.000000,0.000768,0.002627,0.000000,0.000000,...,0.0,0.000000,0.0,0.002664,0.000000,0.003533,0.000000,0.000000,0.0,0.0
Jumanji,0.006275,1.000000,0.008525,0.000000,0.000000,0.010597,0.000000,0.004528,0.012457,0.002581,...,0.0,0.000000,0.0,0.000000,0.001709,0.000000,0.015249,0.000000,0.0,0.0
Grumpier Old Men,0.002190,0.008525,1.000000,0.002432,0.012527,0.000000,0.002171,0.002810,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.003495,0.001365,0.000000,0.000000,0.0,0.0
Waiting to Exhale,0.000860,0.000000,0.002432,1.000000,0.001089,0.002084,0.002418,0.003150,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.003094,0.011325,0.001157,0.000748,0.0,0.0
Father of the Bride Part II,0.000914,0.000000,0.012527,0.001089,1.000000,0.000000,0.011647,0.000000,0.006918,0.000000,...,0.0,0.000000,0.0,0.000000,0.004134,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Robin Hood: Yet another version of the classic epic, with enough variation...",0.003533,0.000000,0.001365,0.011325,0.000000,0.001558,0.001357,0.001765,0.002293,0.001835,...,0.0,0.000000,0.0,0.005732,0.001082,1.000000,0.001045,0.002409,0.0,0.0
Century of Birthing,0.000000,0.015249,0.000000,0.001157,0.000000,0.000850,0.000000,0.000963,0.000000,0.011413,...,0.0,0.000000,0.0,0.000000,0.000000,0.001045,1.000000,0.001314,0.0,0.0
"Betrayal: When one of her hits goes wrong, a professional assassin...",0.000000,0.000000,0.000000,0.000748,0.000000,0.006492,0.000000,0.002220,0.013823,0.004435,...,0.0,0.002352,0.0,0.000000,0.007331,0.002409,0.001314,1.000000,0.0,0.0


In [228]:
for movie in item_similarity:
    print(movie)
    break

Toy Story


In [229]:
my_dict={}
for movie in item_similarity:
    resul_recomen=item_similarity[movie].sort_values(ascending=False)[1:6]
    a=resul_recomen.index.tolist()
    nuevo_elemento = {movie:a}
    my_dict.update(nuevo_elemento)

In [230]:
df = pd.DataFrame(list(my_dict.items()), columns=['Movie', 'Recomendacion'])
df

,Movie,Recomendacion
0,Toy Story,"[Toy Story 2, Toy Story 3, Toy Story of Terror..."
1,Jumanji,"[Game Over: ......, Liar Game: Reborn, Table N..."
2,Grumpier Old Men,"[Grumpy Old Men, Bait Shop, The Hundred-Foot J..."
3,Waiting to Exhale,"[Waiting for Forever, The Carriers Are Waiting..."
4,Father of the Bride Part II,[Father of the Bride: George Banks is an ordin...
...,...,...
44341,Robin Hood: Yet another version of the classic...,[Cinderella: Updated version of the classic Ro...
44342,Century of Birthing,"[The Dark Stranger, Century, The Painting, Nec..."
44343,"Betrayal: When one of her hits goes wrong, a p...","[Naked Killer, Crank, The Eiger Sanction, It's..."
44344,Satan Triumphant,[Sweet Sixteen: A beautiful lonely girl named ...


In [233]:
df.to_csv("../DatosProcesados//recomendation.csv", index=False) 

In [231]:
def obtener_recomendaciones_por_titulo(titulo:str):
    filtro = df['Movie'] == titulo
    if filtro.any():  # Verificar si hay al menos una coincidencia en el título
        recomendaciones = df.loc[filtro, 'Recomendacion'].values
        return recomendaciones[0]
    else:
        return None  # No se encontró el título en la lista


In [232]:
obtener_recomendaciones_por_titulo('Toy Story')

['Toy Story 2',
 'Toy Story 3',
 'Toy Story of Terror!',
 'Small Fry',
 'Toy Story That Time Forgot']

#Otra función que hace lo mismo

In [184]:
def recomendacion(title:str):
    if title not in item_similarity:
        resul_recomen =("El título no se encuentra en la base de datos.")
        return resul_recomen
    else:
        resul_recomen=item_similarity[title] #.sort_values(ascending=False)[1:6]
        r=resul_recomen.nlargest(6)[1:].index.tolist()
    return r

In [191]:
df_movies['Recomendacion']=df_movies['title'].apply(recomendacion)

In [192]:
df_movies

,index,title,genres,overview,combinado,Recomendacion
0,0,Toy Story,"Animation, Comedy, Family","Led by Woody, Andy's toys live happily in his ...",Animation Comedy Family Toy Story Led by Woody...,"[Toy Story 2, Toy Story 3, Rebel Without a Cau..."
1,1,Jumanji,"Adventure, Fantasy, Family",When siblings Judy and Peter discover an encha...,Adventure Fantasy Family Jumanji When siblings...,[Nine Lives: A stuffy businessman finds himsel...
2,5,"Heat: Obsessive master thief, Neil McCauley le...","Action, Crime, Drama, Thriller","Obsessive master thief, Neil McCauley leads a ...",Action Crime Drama Thriller Heat Obsessive mas...,"[The Town, Ant-Man, Stolen: A former thief fra..."
3,9,GoldenEye,"Adventure, Action, Thriller",James Bond must unmask the mysterious head of ...,Adventure Action Thriller GoldenEye James Bond...,"[Octopussy, Licence to Kill, Live and Let Die,..."
4,12,Balto,"Family, Animation, Adventure",An outcast half-wolf risks his life to prevent...,Family Animation Adventure Balto An outcast ha...,"[The Dictator, The Fourth Kind, The Jungle Boo..."
...,...,...,...,...,...,...
3418,43865,2:22: A man's life is derailed when an ominous...,"Drama, Thriller",A man's life is derailed when an ominous patte...,Drama Thriller 222 A mans life is derailed whe...,[Derailed: When two married business executive...
3419,43919,"The Dark Tower: The last Gunslinger, Roland De...","Action, Western, Science Fiction, Fantasy, Horror","The last Gunslinger, Roland Deschain, has been...",Action Western Science Fiction Fantasy Horror ...,"[High-Rise, Tangled: When the kingdom's most w..."
3420,44037,The Emoji Movie,"Comedy, Family, Animation","Gene, a multi-expressional emoji, sets out on ...",Comedy Family Animation The Emoji Movie Gene a...,"[Ruby Red, Disaster Movie, The Princess and th..."
3421,44070,Shot Caller,"Drama, Crime, Thriller",A newly-released prison gangster is forced by ...,Drama Crime Thriller Shot Caller A newlyreleas...,"[West Side Story, Beverly Hills Cop III, Londo..."
